In [ ]:
!pip show pydantic-ai

In [ ]:
!pip install -U pydantic-ai

In [ ]:
from google.colab import userdata
import os
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

- Use Groq as LLM provider in Pydantic-AI

https://ai.pydantic.dev/models/groq/


- Agents Introduction in Pydantic-AI

https://ai.pydantic.dev/agents/#introduction

# Simple Agent

In [ ]:
from pydantic_ai import Agent

In [ ]:


agent = Agent('groq:openai/gpt-oss-120b', system_prompt="You answer questions only related to sports")

In [ ]:
result = await agent.run('Where does "hello world" come from?')


In [ ]:
result.output

'I’m sorry, but I can only help with questions about sports.'

In [ ]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='You answer questions only related to sports', timestamp=datetime.datetime(2025, 12, 9, 5, 6, 55, 150523, tzinfo=datetime.timezone.utc)), UserPromptPart(content='Where does "hello world" come from?', timestamp=datetime.datetime(2025, 12, 9, 5, 6, 55, 150536, tzinfo=datetime.timezone.utc))], run_id='c1364a1d-990e-4721-a477-4562251c37ed'),
 ModelResponse(parts=[ThinkingPart(content='The user asks a question unrelated to sports. The developer instruction says "You answer questions only related to sports". So we must refuse or give a brief apology and state we can only answer sports-related queries. According to policy, we can politely decline.'), TextPart(content='I’m sorry, but I can only help with questions about sports.')], usage=RequestUsage(input_tokens=90, output_tokens=74), model_name='openai/gpt-oss-120b', timestamp=datetime.datetime(2025, 12, 9, 5, 6, 55, tzinfo=TzInfo(0)), provider_name='groq', provider_details={'finish_reason': 'sto

Providing message history to the agent

In [ ]:
result = await agent.run("When did India won the world cup in cricket?")

In [ ]:
result.new_messages()

[ModelRequest(parts=[SystemPromptPart(content='You answer questions only related to sports', timestamp=datetime.datetime(2025, 12, 9, 5, 8, 19, 677422, tzinfo=datetime.timezone.utc)), UserPromptPart(content='When did India won the world cup in cricket?', timestamp=datetime.datetime(2025, 12, 9, 5, 8, 19, 677434, tzinfo=datetime.timezone.utc))], run_id='118711a7-0130-484c-adcd-d1a1c2ac885f'),
 ModelResponse(parts=[ThinkingPart(content='The user asks: "When did India won the (sic) the world cup in cricket?" This is sports related. Provide answer. Provide years: 1983 (won under Kapil Dev), 2011 (under MS Dhoni). Also maybe T20 World Cup 2021, 2022? The question likely about ICC Cricket World Cup (ODI). Provide dates. Also could mention women\'s? But ask India won the world cup in cricket. So answer: 1983 and 2011. Provide details.'), TextPart(content='India has lifted the ICC\u202fCricket World Cup (the 50‑over\u202fODI tournament) **twice**:\n\n| Year | Host nation(s) | Final opponent | 

In [ ]:
await agent.run("What was my last question?", message_history = result.new_messages())

AgentRunResult(output='Your most recent question was:\n\n**“When did India won the world cup in cricket?”**')

# Agent with Structured Response


This example shows how to get structured, type-safe responses from the agent.

Key concepts:
- Using Pydantic models to define response structure
- Type validation and safety


In [ ]:
from pydantic import BaseModel, Field

model = 'groq:openai/gpt-oss-120b'

class ResponseModel(BaseModel):
    """Structured response with metadata."""

    response: str
    needs_escalation: bool
    follow_up_required: bool
    sentiment: str = Field(description="Customer sentiment analysis")


agent2 = Agent(
    model=model,
    output_type=ResponseModel,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses."
    ),
)

response = await agent2.run("How can I track my order #12345?")


In [ ]:
print(response.output.response)

Sure! To track your order #12345, you can:
1. Visit our website and log into your account.
2. Go to the 'My Orders' section and locate order #12345.
3. Click the 'Track Order' button to view the latest shipping status and carrier details.
Alternatively, you can use the direct tracking link we’ve sent to your email, or call our support line at 1-800-555-0123 and provide the order number for real‑time updates.
If you have any trouble accessing the tracking information, just let us know and we’ll help further.


# Agentic RAG

In [ ]:
import os
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [ ]:
from pydantic_ai import Agent
from typing import List

In [ ]:
# Define a retriever tool for RAG

def retrieve_docs(query: str) -> List[str]:
    """
    Retrieve relevant documents for a query.
    In real life, call your vector DB / search index here.
    """
    # TODO: replace this with your real retrieval
    fake_corpus = {
        "pydantic": "Pydantic is a library for data validation using Python type hints.",
        "rag": "RAG stands for Retrieval Augmented Generation.",
        "agent": "Agents can call tools to fetch external information."
    }
    return [text for key, text in fake_corpus.items() if key in query.lower()]


In [ ]:
# Create the agent and attach the tool
rag_agent = Agent(
    "groq:openai/gpt-oss-120b",
    system_prompt=(
        "You are a RAG assistant.\n"
        "- Use the `retrieve_docs` tool whenever user questions may require external info.\n"
        "- When you call it, read the returned documents and answer using ONLY that info plus the question.\n"
        "- If the tool returns nothing, say you couldn't find anything relevant."
    ),
    tools=[retrieve_docs],
)


In [ ]:
question = "Explain what RAG is and how it relates to agents."

In [ ]:
result = await rag_agent.run(question)
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content="You are a RAG assistant.\n- Use the `retrieve_docs` tool whenever user questions may require external info.\n- When you call it, read the returned documents and answer using ONLY that info plus the question.\n- If the tool returns nothing, say you couldn't find anything relevant.", timestamp=datetime.datetime(2025, 12, 9, 5, 18, 10, 802835, tzinfo=datetime.timezone.utc)), UserPromptPart(content='Explain what RAG is and how it relates to agents.', timestamp=datetime.datetime(2025, 12, 9, 5, 18, 10, 802847, tzinfo=datetime.timezone.utc))], run_id='9e438838-f54d-457c-95c1-0b5bf0808fb5'),
 ModelResponse(parts=[ThinkingPart(content='The user asks: "Explain what RAG is and how it relates to agents."\n\nWe need to answer. This is general knowledge; we may not need external docs. However the instructions say: "Use the retrieve_docs tool whenever user questions may require external info." This is a conceptual question that we can answer from our ow

In [ ]:
# Run a RAG-style query

async def ask(question: str):
    result = await rag_agent.run(question)
    print(result.output)


In [ ]:
await ask(question)


RAG stands for **Retrieval Augmented Generation**. It’s a pattern where a language model first retrieves relevant external information and then uses that information to generate its response.  

In the context of AI **agents**, the same idea appears when an agent is given the ability to call tools (such as a search or database lookup) to fetch up‑to‑date or specialized data before producing an answer. The tool‑calling capability of agents is essentially the “retrieval” step of RAG, and the subsequent language‑model generation is the “augmented generation” step. Thus, RAG describes the overall workflow, while agents are the entities that can execute that workflow by invoking retrieval tools and then generating responses.


### Fetch which tool is called

In [ ]:
from pydantic_ai.messages import ModelResponse, ToolCallPart

def get_tools_called(result):
    tools_called = []

    for msg in result.all_messages():
        if isinstance(msg, ModelResponse):
            for part in msg.parts:
                if isinstance(part, ToolCallPart):
                    tools_called.append(part.tool_name)

    return tools_called

get_tools_called(result)

['retrieve_docs']

# Example: Evaluating a RAG application (retrieval + answer quality)

Here we combine:

A Pydantic-AI RAG agent using a search_docs tool (very similar to the official Pydantic RAG example).


Pydantic-Evals to:

1. Check that the answer is grounded in retrieved docs

2. Check that the answer is relevant to the question

- RAG agent skeleton

Assume you already have an in-memory document store and a simple retrieval function.

In [ ]:
from dataclasses import dataclass
from typing import List
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext

# --- Domain models ----------------------------------------------------

class DocChunk(BaseModel):
    id: str
    text: str

@dataclass
class RAGDeps:
    # could be a vector store or just a list of docs in a PoC
    documents: List[DocChunk]

# --- Retrieval tool ---------------------------------------------------

def retrieve_relevant_chunks(query: str, docs: List[DocChunk], k: int = 3) -> List[DocChunk]:
    # toy implementation: top-k by simple keyword overlap
    scores = []
    query_terms = set(query.lower().split())
    for d in docs:
        overlap = len(query_terms & set(d.text.lower().split()))
        scores.append((overlap, d))
    scores.sort(reverse=True, key=lambda t: t[0])
    return [d for score, d in scores[:k] if score > 0]

# --- RAG agent --------------------------------------------------------

class RAGAnswer(BaseModel):
    answer: str
    used_doc_ids: List[str]

rag_agent = Agent[RAGDeps, RAGAnswer](
    "openai:gpt-4o-mini",
    deps_type=RAGDeps,
    output_type=RAGAnswer,
    instructions="""
    You are a documentation assistant.
    Use ONLY the provided context chunks to answer the question.
    If the answer is not in the context, say you don't know.
    Return the IDs of the chunks you used.
    """,
)

@rag_agent.tool  # function tool exposes retrieval to the model
def search_docs(ctx: RunContext[RAGDeps], query: str) -> List[DocChunk]:
    return retrieve_relevant_chunks(query, ctx.deps.documents)


- Task function for evals

We want the task function to return both answer text and which docs were used:

In [ ]:
from typing import TypedDict, List

class RAGOutput(TypedDict):
    answer: str
    used_doc_ids: List[str]

def rag_task(question: str, deps: RAGDeps) -> RAGOutput:
    result = rag_agent.run_sync(question, deps=deps)
    out = result.output
    return {"answer": out.answer, "used_doc_ids": out.used_doc_ids}


For simple evals we can partially “fix” dependencies (e.g., same KB for all cases):

In [ ]:
# Partial application to match the expected signature inputs -> output
my_docs = [
    DocChunk(id="d1", text="Our refund policy allows returns within 30 days."),
    DocChunk(id="d2", text="Tech support is available 24/7 via chat."),
]

def rag_task_fixed(question: str) -> RAGOutput:
    return rag_task(question, deps=RAGDeps(documents=my_docs))


- Define RAG-specific eval dataset

Key idea: in each case we specify both:

1. The question

2. The expected supporting doc IDs

3. Optionally, a reference answer

In [ ]:
from pydantic_evals import Case, Dataset
from pydantic_evals.evaluators import LLMJudge

class RAGInputs(BaseModel):
    question: str
    expected_doc_ids: List[str]
    reference_answer: str

dataset = Dataset[RAGInputs, RAGOutput](
    cases=[
        Case(
            name="refund_policy",
            inputs=RAGInputs(
                question="What is your refund policy?",
                expected_doc_ids=["d1"],
                reference_answer="We allow refunds within 30 days of purchase.",
            ),
        ),
        Case(
            name="support_hours",
            inputs=RAGInputs(
                question="When is tech support available?",
                expected_doc_ids=["d2"],
                reference_answer="Tech support is available 24/7 via chat.",
            ),
        ),
    ],
)


- Add custom evaluators: retrieval + groundedness

You can layer:

Retrieval precision: Did the agent use the right doc IDs?

Groundedness: Is the answer consistent with provided docs?
(Good use case for LLMJudge.)


In [ ]:
### Retrieval evaluator (deterministic)


from dataclasses import dataclass
from pydantic_evals.evaluators import Evaluator, EvaluatorContext, EvaluationReason

@dataclass
class RetrievalMatch(Evaluator[RAGInputs, RAGOutput, None]):
    """Check that the agent used all expected docs."""

    def evaluate(self, ctx: EvaluatorContext[RAGInputs, RAGOutput, None]):
        expected = set(ctx.inputs.expected_doc_ids)
        used = set(ctx.output["used_doc_ids"])
        missing = expected - used
        extra = used - expected

        ok = not missing  # require at least all expected docs
        reason = f"missing={missing}, extra={extra}"
        return EvaluationReason(value=ok, reason=reason)


Attach this evaluator to the dataset:

In [ ]:
dataset.add_evaluator(RetrievalMatch())

(You can also attach evaluators to specific cases by specific_case="refund_policy".)

In [ ]:
### Grounded answer evaluator with LLMJudge


from pydantic_evals.evaluators import LLMJudge

groundedness_rubric = """
You are evaluating an answer produced by a RAG system.

Criteria (True vs False):
- TRUE if the answer is fully supported by the provided reference_answer text
  and does not introduce any contradictions or extra facts.
- FALSE if the answer contradicts, fabricates, or goes beyond the reference.
"""

dataset.add_evaluator(
    LLMJudge(
        rubric=groundedness_rubric,
        include_input=True,            # see the question + inputs
        include_expected_output=True,  # see reference_answer
        # model optional – uses default judge model, typically `openai:gpt-4o`
    )
)


### Under the hood, LLMJudge will call a judge model and return a boolean/score with an explanation.

Run RAG evaluation

In [ ]:
report = dataset.evaluate_sync(rag_task_fixed)
report.print()

# Example: Evaluating a Agent with Pydantic-Evals

- Agent we want to evaluate

A minimal Pydantic-AI agent that classifies user queries into support intents:

In [ ]:
from typing import Literal
from pydantic import BaseModel
from pydantic_ai import Agent

class IntentOutput(BaseModel):
    intent: Literal["refund", "technical_support", "sales", "other"]
    reasoning: str

# Pydantic AI agent: takes a user query and returns typed output
intent_agent = Agent[None, IntentOutput](
    "openai:gpt-4o-mini",  # or any configured model
    instructions="""
    You are a support triage bot.
    Classify the user's message into one of: refund, technical_support, sales, other.
    Explain your reasoning in one or two sentences.
    """,
    output_type=IntentOutput,
)


This is straight out of typical Pydantic-AI usage: agents are parameterized by dependency type and output model, and output_type is a Pydantic model that gets validated


We’ll now evaluate whether the classification is correct across a dataset.

- Wrap the agent as a task function

Pydantic-Evals expects a function to call for each test case. We wrap the agent:

In [ ]:
def classify_intent_task(user_message: str) -> str:
    """Task function for evals – returns only the intent label."""
    result = intent_agent.run_sync(user_message)
    return result.output.intent


- Define dataset + evaluators (deterministic + LLM-judge)

In [ ]:
from pydantic_evals import Case, Dataset
from pydantic_evals.evaluators import EqualsExpected, LLMJudge

# 1) Dataset with labeled cases
dataset = Dataset[str, str](
    cases=[
        Case(
            name="refund_request",
            inputs="I want a refund for the headphones I bought last week.",
            expected_output="refund",
        ),
        Case(
            name="login_issue",
            inputs="I can't log into my account, it keeps saying password invalid.",
            expected_output="technical_support",
        ),
        Case(
            name="pricing_question",
            inputs="Do you offer any discount if we buy 100 licenses?",
            expected_output="sales",
        ),
    ],
    evaluators=[
        # Exact correctness: output == expected_output
        EqualsExpected(),
        # Subjective check: was the classification reasonable?
        LLMJudge(
            rubric=(
                "Judge if the predicted intent label is a reasonable "
                "classification for the given user message. "
                "Return true only if it clearly fits."
            ),
            include_input=True,
            include_expected_output=True,
            # model optional – uses default judge model if omitted
        ),
    ],
)


- EqualsExpected is a built-in comparison evaluator.


- LLMJudge is an LLM-as-a-judge evaluator for subjective criteria (correctness of label given context).

In [ ]:
### Run the evaluation


report = dataset.evaluate_sync(classify_intent_task)

# Pretty print high-level summary in training session:
report.print()


Pydantic-Evals’ data model is:

Dataset – list of Cases + Evaluators

Experiment – running dataset.evaluate(task)

EvaluationReport – structured result with scores, assertions, durations, etc.

# Performing Evaluations on AI Agent using library called as DeepEval

In [ ]:
!pip show deepeval

In [ ]:
!pip install deepeval

#### Configuring deepeval

In [ ]:
### Login to Confident AI

import deepeval
deepeval.login("")


🎉🥳 Congratulations! You've successfully logged in! 🙌

In [ ]:
### if you want to use Groq API as you LLM provider for Deepeval then please perform the below configuation

import os
os.environ["OPENAI_BASE_URL"] = "https://api.groq.com/openai/v1"  # Groq’s OpenAI-compatible API
os.environ["OPENAI_API_KEY"] = ""               # use your Groq key here

In [ ]:
### Set Groq model as local model in Deepeval

!deepeval set-local-model --model-name="openai/gpt-oss-120b" --base-url="https://api.groq.com/openai/v1" --api-key=""

Settings updated for this session. To persist, use --save=dotenv[:path] (default
.env.local) or set DEEPEVAL_DEFAULT_SAVE=dotenv:.env.local
🙌 Congratulations! You're now using a local model `openai/gpt-oss-120b` for all
evals that require an LLM.


### Testing AI Agent

In [ ]:
### Create a test case - Here we are checking weather the correct tool was called for a giver user question


from deepeval.test_case import LLMTestCase
from deepeval.test_case import ToolCall

test_case = LLMTestCase(

    input=question,             ### User question

    tools_called=[ToolCall(name=get_tools_called(result)[-1])],                ### Tool called by the agent

    actual_output=result.output,    ### Agent response

    expected_output=result.output,  ### Ground truth answer

    expected_tools=[ToolCall(name = 'retrieve_docs')] ###Expected tool to be called Ground truth

    )

In [ ]:
print(test_case.input)

print(test_case.actual_output)

print(test_case.expected_output)

print(test_case.tools_called)

print(test_case.expected_tools)

Explain what RAG is and how it relates to agents.
**RAG (Retrieval‑Augmented Generation)** is a technique where a language model first **retrieves** relevant external information and then **generates** its response using that retrieved material.  

**Relation to agents:**  
Agents are designed to **call tools** (such as search or database look‑ups) to obtain up‑to‑date or specialized data. This tool‑calling step is essentially the “retrieval” part of RAG, and the agent’s subsequent reasoning or response generation is the “generation” part. In this way, agents implement RAG by fetching external information before producing their answer.
**RAG (Retrieval‑Augmented Generation)** is a technique where a language model first **retrieves** relevant external information and then **generates** its response using that retrieved material.  

**Relation to agents:**  
Agents are designed to **call tools** (such as search or database look‑ups) to obtain up‑to‑date or specialized data. This tool‑cal

In [ ]:
### Create an evaluation dataset
import deepeval
from deepeval.dataset import EvaluationDataset

In [ ]:
### Add the test case to the evaluation dataset
dataset = EvaluationDataset()
dataset.add_test_case(test_case)

In [ ]:
dataset

EvaluationDataset(test_cases=[LLMTestCase(input='Explain what RAG is and how it relates to agents.', actual_output='**RAG (Retrieval‑Augmented Generation)** is a technique where a language model first **retrieves** relevant external information and then **generates** its response using that retrieved material.  \n\n**Relation to agents:**  \nAgents are designed to **call tools** (such as search or database look‑ups) to obtain up‑to‑date or specialized data. This tool‑calling step is essentially the “retrieval” part of RAG, and the agent’s subsequent reasoning or response generation is the “generation” part. In this way, agents implement RAG by fetching external information before producing their answer.', expected_output='**RAG (Retrieval‑Augmented Generation)** is a technique where a language model first **retrieves** relevant external information and then **generates** its response using that retrieved material.  \n\n**Relation to agents:**  \nAgents are designed to **call tools** (s

In [ ]:
dataset.goldens

[]

In [ ]:
### We want to evaluate the tool correctness metric on this test case

from deepeval.metrics import ToolCorrectnessMetric

In [ ]:
os.environ["LOCAL_MODEL_API_KEY"] = os.environ["OPENAI_API_KEY"]
os.environ["LOCAL_MODEL_BASE_URL"] = "https://api.groq.com/openai/v1"
os.environ["LOCAL_MODEL_NAME"] = "openai/gpt-oss-120b"

In [ ]:
### Evaluate testcases w.r.t predefined metrics ToolCorrectnessMetric()

deepeval.evaluate(dataset.test_cases, metrics= [ToolCorrectnessMetric()])

✨ You're running DeepEval's latest Tool Correctness Metric! (using None, strict=False, async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases




Metrics Summary

  - ✅ Tool Correctness (score: 1.0, threshold: 0.5, strict: False, evaluation model: None, reason: [
	 Tool Calling Reason: All expected tools ['retrieve_docs'] were called (order not considered).
	 Tool Selection Reason: No available tools were provided to assess tool selection criteria
]
, error: None)

For test case:

  - input: Explain what RAG is and how it relates to agents.
  - actual output: **RAG (Retrieval‑Augmented Generation)** is a technique where a language model first **retrieves** relevant external information and then **generates** its response using that retrieved material.  

**Relation to agents:**  
Agents are designed to **call tools** (such as search or database look‑ups) to obtain up‑to‑date or specialized data. This tool‑calling step is essentially the “retrieval” part of RAG, and the agent’s subsequent reasoning or response generation is the “generation” part. In this way, agents implement RAG by fetching external information before producin

⚠ WARNING: No hyperparameters logged.
» ]8;id=660831;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Done 🎉! View results on 
]8;id=3159;https://app.confident-ai.com/project/cmix2a74i00pvpm1fyu5zhour/test-runs/cmiy8rubb0juakc1f8kfmln5f/regression-testing\https://app.confident-ai.com/project/cmix2a74i00pvpm1fyu5zhour/test-runs/cmiy8rubb0juakc1f8kfmln5f/regression-testi]8;;\
]8;id=3159;https://app.confident-ai.com/project/cmix2a74i00pvpm1fyu5zhour/test-runs/cmiy8rubb0juakc1f8kfmln5f/regression-testing\ng]8;;\

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Tool Correctness', threshold=0.5, success=True, score=1.0, reason="[\n\t Tool Calling Reason: All expected tools ['retrieve_docs'] were called (order not considered).\n\t Tool Selection Reason: No available tools were provided to assess tool selection criteria\n]\n", strict_mode=False, evaluation_model=None, error=None, evaluation_cost=0.0, verbose_logs='Expected Tools:\n[\n    ToolCall(\n        name="retrieve_docs"\n    )\n] \n \nTools Called:\n[\n    ToolCall(\n        name="retrieve_docs"\n    )\n] \n \nAvailable Tools: [] \n \nTool Selection Score: 1.0 \n \nTool Selection Reason: No available tools were provided to assess tool selection criteria')], conversational=False, multimodal=False, input='Explain what RAG is and how it relates to agents.', actual_output='**RAG (Retrieval‑Augmented Generation)** is a technique where a language model first **retrieves** relevant external